-------------------------------------------------------

**Clasificación de videojuegos por popularidad**

*Juan Mancera López*

-----

Iniciamos cargando las bibliotecas necesarias, cargando el dataset y mostrando un poco de la información general que este contiene

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# Cargar el dataset
df = pd.read_csv("vgchartz-2024.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   img           64016 non-null  object 
 1   title         64016 non-null  object 
 2   console       64016 non-null  object 
 3   genre         64016 non-null  object 
 4   publisher     64016 non-null  object 
 5   developer     63999 non-null  object 
 6   critic_score  6678 non-null   float64
 7   total_sales   18922 non-null  float64
 8   na_sales      12637 non-null  float64
 9   jp_sales      6726 non-null   float64
 10  pal_sales     12824 non-null  float64
 11  other_sales   15128 non-null  float64
 12  release_date  56965 non-null  object 
 13  last_update   17879 non-null  object 
dtypes: float64(6), object(8)
memory usage: 6.8+ MB


Eliminamos los elementos que no poseen puntuación crítica o conteo de ventas ya que dichos datos serán ocupados para medir su popularidad

In [2]:
# Eliminar vacíos 
df = df.dropna(subset=["critic_score"])
df = df.dropna(subset=["total_sales"])

Creamos otro dataframe para juntar los juegos que están divididos por diferentes plataformas, las ventas totales las sumamos y en la puntuación crítica generamos un promedio de las que haya.

In [3]:
# Creamos nuevo dataframe con los datos juntados
dataframe = df.groupby("title").agg({"total_sales": "sum", "critic_score": "mean"})
print(dataframe)

                               total_sales  critic_score
title                                                   
.hack//G.U. Vol.2//Reminisce          0.23         6.200
.hack//G.U. Vol.3//Redemption         0.17         5.700
.hack//Infection Part 1               1.27         7.700
.hack//Mutation Part 2                0.68         7.500
.hack//Outbreak Part 3                0.46         7.100
...                                    ...           ...
bit Generations: Dotstream            0.01         8.000
de Blob 2                             0.65         7.725
echochrome                            0.09         7.600
echoshift                             0.00         7.500
inFAMOUS 2                            1.87         8.100

[2876 rows x 2 columns]


En este caso para clasificarlos decidí usar los cuartiles al 33% y 66% para así que se repartan uniformemente los resultados. Esto lo hacemos para ventas y calificaciones

In [4]:
# Definir percentiles para la clasificación de las ventas
q1_sales = dataframe["total_sales"].quantile(0.33)  # El 33%
q2_sales = dataframe["total_sales"].quantile(0.66)  # El 66%

# Definir percentiles para la clasificación de las calificaciones
q1_score = dataframe["critic_score"].quantile(0.33)
q2_score = dataframe["critic_score"].quantile(0.66)

Generamos una función creada arbitrariamente para generar su categoría de popularidad. 

_"Muy popular"_ Cuando esté por encima de los segundos cuartiles tanto de ventas como de calificación

_"Moderadamente popular"_ Cuando esté por encima de los primeros cuartiles tanto de ventas como de calificación

Y _"Menos popular"_ En cualquier otro caso

Usamos esta función para generar la columna de _"popularity"_ (popularidad). Para ello usamos la función apply, se usa axis=1 para que se genere en una nueva columna.

Además, este dataframe con la nueva columna lo guardo en un nuevo archivo para poder revisarlo de manera completa.

In [5]:
def clasificar_popularidad(row):
    if row["total_sales"] >= q2_sales and row["critic_score"] >= q2_score:
        return "Muy popular"
    elif row["total_sales"] >= q1_sales and row["critic_score"] >= q1_score:
        return "Moderadamente popular"
    else:
        return "Menos popular"
    
dataframe["popularity"] = dataframe.apply(clasificar_popularidad, axis=1)
dataframe.to_csv("Resultados.csv")

Ahora separamos los datos, poniendo como datos la critica y las ventas, y como objetivo la popularidad.

También separamos entre datos de entrenamiento y datos de prueba, utilizando un 70% de los datos para el entrenamiento.

In [6]:
x = dataframe[["critic_score", "total_sales"]]
y = dataframe["popularity"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=5)

Indicamos que el modelo es de tipo _"Random forest"_ con 20 árboles y una profundidad máxima de 5. Originalmente era con 200 árboles, pero se vió que daba el mismo resultado con 20 árboles.

Entrenamos el modelo con los datos de entrenamiento.

In [7]:
model = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=5)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=20, random_state=5)

Y ahora hacemos una prediccion con los datos de prueba. 

Posteriormente verificamos su exactitud y un reporte de la clasificación.

In [8]:
y_prediction = model.predict(x_test)
print("Exactitud: ", accuracy_score(y_test, y_prediction))
print("Reporte de clasificación:\n", classification_report(y_test, y_prediction))

Exactitud:  1.0
Reporte de clasificación:
                        precision    recall  f1-score   support

        Menos popular       1.00      1.00      1.00       435
Moderadamente popular       1.00      1.00      1.00       288
          Muy popular       1.00      1.00      1.00       140

             accuracy                           1.00       863
            macro avg       1.00      1.00      1.00       863
         weighted avg       1.00      1.00      1.00       863



Vemos que su exactitud es exacta, esto se debe probablemente a que la popularidad se construye a partir de reglas muy básicas de sus ventas y calificaciones, es por ello que no le ha costado construir un modelo que sea capaz de predecir correctamente la popularidad de cada videojuego.

In [9]:
dataframe["popularity"].value_counts()

popularity
Menos popular            1455
Moderadamente popular     943
Muy popular               478
Name: count, dtype: int64

Vemos la cantidad que fue puesta en cada categoría de su popularidad.

-----

A continuación decidí volver a intentarlo escalando las ventas y la calificación antes de entrenarla.

In [10]:
from sklearn.preprocessing import StandardScaler

dataframe_sc = dataframe[["total_sales","critic_score"]]

numeric_column = dataframe_sc.select_dtypes(include=['float64', 'int64']).columns

scaler = StandardScaler()
dataframe_sc[numeric_column] = scaler.fit_transform(dataframe_sc[numeric_column])
dataframe_sc


C:\Users\juanm\AppData\Local\Temp\ipykernel_3896\4264449562.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_sc[numeric_column] = scaler.fit_transform(dataframe_sc[numeric_column])


,total_sales,critic_score
title,,
.hack//G.U. Vol.2//Reminisce,-0.332750,-0.557655
.hack//G.U. Vol.3//Redemption,-0.356873,-0.902039
.hack//Infection Part 1,0.085372,0.475495
.hack//Mutation Part 2,-0.151832,0.337741
.hack//Outbreak Part 3,-0.240281,0.062235
...,...,...
bit Generations: Dotstream,-0.421199,0.682125
de Blob 2,-0.163893,0.492714
echochrome,-0.389036,0.406618


Creamos sus respectivos cuartiles.

In [11]:
# Definir percentiles para la clasificación de las ventas
q1_sales_sc = dataframe_sc["total_sales"].quantile(0.33)  # El 33%
q2_sales_sc = dataframe_sc["total_sales"].quantile(0.66)  # El 66%

# Definir percentiles para la clasificación de las calificaciones
q1_score_sc = dataframe_sc["critic_score"].quantile(0.33)
q2_score_sc = dataframe_sc["critic_score"].quantile(0.66)

Y generamos su propia función, su propia clasificación, dividimos los datos, entrenamos el modelo con los nuevos datos y hacemos una predicción, en este caso usamos el mismo random state para probar en las mismas condiciones.

In [12]:
def clasificar_popularidad_sc(row):
    if row["total_sales"] >= q2_sales_sc and row["critic_score"] >= q2_score_sc:
        return "Muy popular"
    elif row["total_sales"] >= q1_sales_sc and row["critic_score"] >= q1_score_sc:
        return "Moderadamente popular"
    else:
        return "Menos popular"
    
dataframe_sc["popularity"] = dataframe_sc.apply(clasificar_popularidad_sc, axis=1)
dataframe_sc.to_csv("Resultados_2.csv")

x = dataframe_sc[["critic_score", "total_sales"]]
y = dataframe_sc["popularity"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=5)

model = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=5)
model.fit(x_train, y_train)

y_prediction = model.predict(x_test)
print("Exactitud: ", accuracy_score(y_test, y_prediction))
print("Reporte de clasificación:\n", classification_report(y_test, y_prediction))

Exactitud:  1.0
Reporte de clasificación:
                        precision    recall  f1-score   support

        Menos popular       1.00      1.00      1.00       435
Moderadamente popular       1.00      1.00      1.00       288
          Muy popular       1.00      1.00      1.00       140

             accuracy                           1.00       863
            macro avg       1.00      1.00      1.00       863
         weighted avg       1.00      1.00      1.00       863



Vemos que tiene unos resultados muy similares.

In [13]:
dataframe_sc["popularity"].value_counts()

popularity
Menos popular            1455
Moderadamente popular     943
Muy popular               478
Name: count, dtype: int64

Para finalizar vemos un conteo de la cantidad de cada categoría de popularidad.

In [14]:
iguales = (dataframe_sc["popularity"] == dataframe["popularity"]).sum()
diferentes = len(dataframe_sc) - iguales

print(f"Valores iguales: {iguales}")
print(f"Valores diferentes: {diferentes}")

Valores iguales: 2876
Valores diferentes: 0


Y vemos que ambas clasificaron de la misma manera a los juegos conforme a su popularidad.